In [1]:
import numpy as np
import tensorflow as tf
import time
import os
import math
import pandas as pd
from tqdm.notebook import tqdm
import scipy.sparse as sp
from tffm import TFFMClassifier
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics import roc_auc_score, accuracy_score

dir_ = '../data/'
folder = 'sp_matrix_pop'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site

In [2]:
df = pd.read_pickle(os.path.join(dir_, folder, file_name))
train = pd.read_pickle(os.path.join(dir_, folder, 'train_' + file_name))
train_ = train[['uid', 'tid', 'rating', 'count','pop']]

In [3]:
train_x = sp.load_npz(os.path.join(dir_, folder, 'train_x_' + file_name + '.npz'))
train_y = np.loadtxt(os.path.join(dir_, folder, 'train_y_' + file_name[:-3] + 'csv'), delimiter=',')
test_x_sp = sp.load_npz(os.path.join(dir_, folder, 'test_x_' + file_name + '.npz'))
test_y_sp = np.loadtxt(os.path.join(dir_, folder, 'test_y_' + file_name[:-3] + 'csv'), delimiter=',')

In [4]:
print(train_x[0,:])

  (0, 66)	1.0
  (0, 19252)	1.0
  (0, 39355)	7.0


In [5]:
unique_users = len(df['uid'].unique())
unique_tracks = len(df['tid'].unique())
l = unique_users + unique_tracks + 1
tid_start = unique_users
tag_start = unique_users + unique_tracks
print ('Tid start position : ' + str(tid_start))
print ('Tag start position : ' + str(tag_start))
print ('Length of vector : ' + str(l))
print('----------------------------------------')
print ('Number of unique users : ' + str(unique_users))
print ('Number of unique tracks : ' + str(unique_tracks))

Tid start position : 218
Tag start position : 39355
Length of vector : 39356
----------------------------------------
Number of unique users : 218
Number of unique tracks : 39137


In [6]:
track_ids = []
for i in range(unique_tracks):
    track_ids.append(i)

all_tracks = pd.DataFrame()
all_tracks['tid'] = track_ids
all_tracks['count'] = 0

In [7]:
x_test = []
for i in tqdm(range(unique_users)):
    user = train_[train_['uid']==i]
    top_n = all_tracks.set_index('tid').add(user.set_index('tid'), fill_value=0).reset_index()
    top_n = top_n[top_n['count']==0]
    top_n['uid'] = i
    top_n = top_n[['uid', 'tid']]
    top_n = top_n.values.tolist()
    x_test.extend(top_n)
    

In [8]:
i = 0
v = []
sp_rows = []
sp_cols = []
for row in tqdm(x_test):
    sp_rows.append(i)
    sp_cols.append(row[0])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row[1])
    v.append(1)
    
    t = df[df['tid']==row[1]].reset_index()
    pop = t['pop'].iloc[0]
    sp_rows.append(i)
    sp_cols.append(tag_start)
    v.append(pop)
    
    i += 1

KeyboardInterrupt: 

In [ ]:
test_x = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(x_test), l), dtype=float)

In [ ]:
test_x

In [ ]:
x_test

In [ ]:
# sp.save_npz(os.path.join(dir_, folder, 'topN_test_x_' + file_name), test_x)

In [9]:
test_x = sp.load_npz(os.path.join(dir_, folder, 'topN_test_x_' + file_name + '.npz'))

In [10]:
print(test_x[1,:])

  (0, 0)	1.0
  (0, 232)	1.0
  (0, 39355)	6.0


In [11]:
order = 3
model = TFFMRegressor(
    order=order, 
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
    n_epochs=10, 
    batch_size=16384,
    init_std=0.00001,
    reg=0.0001,
    input_type='sparse'
)
model.fit(train_x, train_y, show_progress=True)
predictions = model.predict(test_x)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


100%|██████████| 10/10 [00:04<00:00,  2.13epoch/s]


In [12]:
len(predictions), predictions

(8169189,
 array([0.89775634, 0.24803326, 2.035172  , ..., 0.22819729, 0.25323915,
        0.22502193], dtype=float32))

In [13]:
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', predictions, False) 
df2[:10]

,uid,tid,rating
0,0,2,0.897756
1,0,14,0.248033
2,0,15,2.035172
3,0,20,0.487427
4,0,21,0.246802
5,0,24,0.902386
6,0,30,0.376095
7,0,44,0.398055
8,0,46,0.373620
9,0,48,3.233408


In [22]:
# df2.to_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))

In [15]:
predictions = model.predict(test_x_sp)
print('RMSE: {}'.format(math.sqrt(mean_squared_error(test_y_sp, predictions))))
print('MAE: {}'.format(mean_absolute_error(test_y_sp, predictions)))

RMSE: 2.1767507609595196
MAE: 1.656735487931028


In [16]:
predictions = model.predict(train_x)
print('RMSE: {}'.format(math.sqrt(mean_squared_error(train_y, predictions))))
print('MAE: {}'.format(mean_absolute_error(train_y, predictions)))

RMSE: 2.174169631629219
MAE: 1.6516069738133277


In [17]:
predictions = df2

In [18]:
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:10]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6
37,0,24,5,9,3.50,23
46,0,30,5,1,3.50,10
61,0,44,13,2,4.75,10
66,0,46,2,6,1.50,9
68,0,48,4,2,3.00,85


In [19]:
num_user = len(predictions['uid'].unique())
num_user

218

In [20]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.15894495412844042


In [21]:
train.sort_values(by=['uid','tid'])[:10]

,uid,tid,count,tags,rating,pop
0,0,0,2,1,1.50,5
2,0,1,2,2,1.50,10
8,0,3,1,0,0.25,9
10,0,4,5,1,3.50,14
12,0,5,2,4,1.50,28
13,0,6,4,8,3.00,8
14,0,7,5,1,3.50,6
15,0,8,18,4,5.00,21
17,0,9,1,2,0.25,32
20,0,10,1,1,0.25,10
